In [ ]:
from aux_functions import *

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import os
import numpy as np

In [ ]:
def earth_figure(mean=None, std=None, observations=None, kwargs={}, kwargs_contour={}):
    # initialize figure
    fig = plt.figure(figsize=(10, 5))
    projection = ccrs.Robinson()
    trsf = ccrs.PlateCarree()
    ax = plt.axes(projection=projection)
    ax.set_global()
    ax.coastlines()
    ax.gridlines(draw_labels=False, crs=trsf)
    
    # draw std if given
    if std is None:
        ax.stock_img()
    else:
        lon = np.linspace(-180, 180, 300)
        lat = np.linspace(-89, 89, 300)
        ax.contourf(lon, lat, std.reshape(300, 300), transform=ccrs.PlateCarree(), zorder=0, alpha=0.7, **kwargs_contour)
    
    # draw mean and observations if given
    if mean is not None:
        X_mean, y_mean = v_car_to_sph(mean[:, :3], mean[:, 3:])
        q = ax.quiver(
            X_mean[:, 1] * 180 / np.pi, X_mean[:, 0] * 180 / np.pi,
            y_mean[:, 1], y_mean[:, 0],
            **kwargs,
            angles="uv", transform=ccrs.PlateCarree(), zorder=1
        )
        if not kwargs:
            q._init()
            # arguments for obs
            kwargs = dict(scale=q.scale, width=q.width)
    
    kwargs_obs = {**kwargs}
    if "width" in kwargs_obs:
        kwargs_obs["width"] *= 2
    
    if observations is not None:
        X_obs, y_obs = v_car_to_sph(observations[:, :3], observations[:, 3:])
        ax.quiver(
            X_obs[:, 1] * 180 / np.pi, X_obs[:, 0] * 180 / np.pi,
            y_obs[:, 1], y_obs[:, 0],
            angles="uv",
            **kwargs_obs,
            color="r", transform=ccrs.PlateCarree(),
            zorder=2
        )
    
    fig.tight_layout()
    return fig, kwargs

In [ ]:
out_folder = "flat-figures"
folder = os.path.join("blender-data", "outputs")

### Uncertainty color scale

In [ ]:
std_pm = np.array(np.genfromtxt(os.path.join(folder, "ERA5_projm12_kappa=0.5_orbit_0_pred__mercator__std.csv"), delimiter=","))
std_hm = np.array(np.genfromtxt(os.path.join(folder, "ERA5_dfhm12_kappa=0.5_orbit_0_pred__mercator__std.csv"), delimiter=","))

u_min, u_max = 0, max(std_pm.max(), std_hm.max())
levels = np.linspace(u_min, u_max, 50)

kwargs_contour = {"levels": levels, "antialiased": True}

# 6c: ground truth and observations

This one sets `kwargs` so needs to be run first!

In [ ]:
means_file = "ERA5_orbit_0__mercator__mean.csv"
obs_file = "ERA5_orbit_0__mercator__observations.csv"

mean = np.array(np.genfromtxt(os.path.join(folder, means_file), delimiter=","))
observations = np.array(np.genfromtxt(os.path.join(folder, obs_file), delimiter=","))

fig, kwargs = earth_figure(mean=mean, observations=observations, kwargs_contour=kwargs_contour)
fig.savefig(os.path.join(out_folder, "fig6c_flat.pdf"))

# 6a: predictive mean and std (proj. Matern)

In [ ]:
means_file = "ERA5_projm12_kappa=0.5_orbit_0_pred__mercator__mean.csv"
std_file = "ERA5_projm12_kappa=0.5_orbit_0_pred__mercator__std.csv"
obs_file = "ERA5_orbit_0__mercator__observations.csv"

mean = np.array(np.genfromtxt(os.path.join(folder, means_file), delimiter=","))
std = np.array(np.genfromtxt(os.path.join(folder, std_file), delimiter=","))
observations = np.array(np.genfromtxt(os.path.join(folder, obs_file), delimiter=","))

fig, _ = earth_figure(mean=mean, std=std, observations=observations, kwargs=kwargs, kwargs_contour=kwargs_contour)
fig.savefig(os.path.join(out_folder, "fig6a_flat.pdf"))

# 6b: posterior sample (proj. Matern)

In [ ]:
means_file = "ERA5_projm12_kappa=0.5_orbit_0_sample_5__mercator__mean.csv"

mean = np.array(np.genfromtxt(os.path.join(folder, means_file), delimiter=","))
observations = np.array(np.genfromtxt(os.path.join(folder, obs_file), delimiter=","))

fig, _ = earth_figure(mean=mean, kwargs={}, kwargs_contour=kwargs_contour)
fig.savefig(os.path.join(out_folder, "fig6b_flat.pdf"))

# 6d: predicted mean and std (Hodge)

In [ ]:
means_file = "ERA5_dfhm12_kappa=0.5_orbit_0_pred__mercator__mean.csv"
std_file = "ERA5_dfhm12_kappa=0.5_orbit_0_pred__mercator__std.csv"
obs_file = "ERA5_orbit_0__mercator__observations.csv"

mean = np.array(np.genfromtxt(os.path.join(folder, means_file), delimiter=","))
std = np.array(np.genfromtxt(os.path.join(folder, std_file), delimiter=","))
observations = np.array(np.genfromtxt(os.path.join(folder, obs_file), delimiter=","))

fig, _ = earth_figure(mean=mean, std=std, observations=observations, kwargs=kwargs, kwargs_contour=kwargs_contour)
fig.savefig(os.path.join(out_folder, "fig6d_flat.pdf"))

# 6e: posterior sample (Hodge)

In [ ]:
means_file = "ERA5_dfhm12_kappa=0.5_orbit_0_sample_1__mercator__mean.csv"

mean = np.array(np.genfromtxt(os.path.join(folder, means_file), delimiter=","))
observations = np.array(np.genfromtxt(os.path.join(folder, obs_file), delimiter=","))

fig, _ = earth_figure(mean=mean, kwargs={}, kwargs_contour=kwargs_contour)
fig.savefig(os.path.join(out_folder, "fig6e_flat.pdf"))